# Combined ETL Python Code

**Insert Dependecies**

In [1]:
import numpy as np
import pandas as pd

**Import Urbanization Index Data**

In [2]:
urban = "Resources/urban.csv"
urban_df = pd.read_csv(urban)
urban_df = pd.DataFrame(urban_df)

In [3]:
state_new  = {'Alabama': 'AL','Alaska': 'AK','American Samoa': 'AS','Arizona': 'AZ','Arkansas': 'AR',
              'California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District of Columbia': 'DC',
              'Florida': 'FL','Georgia': 'GA','Guam': 'GU','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL',
              'Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME',
              'Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS',
              'Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH',
              'New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND',
              'Northern Mariana Islands':'MP','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA',
              'Puerto Rico': 'PR','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN',
              'Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virgin Islands': 'VI','Virginia': 'VA',
              'Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'
}

urban_df['abbrev'] = urban_df.state.replace(state_new)

In [4]:
urban = urban_df[['abbrev', 'urbanindex']].copy()
urban = urban.rename(columns={'abbrev':'State','urbanindex':'Urban_Index'})

In [5]:
clean_urban = urban.drop(index = [2,37,50,12,42])

In [6]:
clean_urban['Urban_Scale']= pd.qcut(clean_urban['Urban_Index'], 4, 
                                    labels=["Very Rural", "Rural", 
                                            "Urban", "Very Urban"])
clean_urban = clean_urban.round(3)
clean_urban.head()

,State,Urban_Index,Urban_Scale
0,AL,9.606,Very Rural
1,AK,8.736,Very Rural
3,AZ,11.300,Very Urban
4,AR,9.259,Very Rural
5,CA,12.190,Very Urban


**Import Ufo Sighting Data**

In [7]:
ufodata = "Resources/scrubbed_ufo.csv"
ufodf = pd.read_csv(ufodata, low_memory=False)

In [8]:
datetimefix = ufodf['datetime']
datetimefix = pd.DataFrame(datetimefix.str.split("/",expand=True))
datetimefix = datetimefix[2]
datetimefix = pd.DataFrame(datetimefix.str.split(expand=True))
datetimefix = pd.to_numeric(datetimefix[0])
datetimefix = pd.DataFrame(datetimefix)

In [9]:
ufodf = ufodf.join(datetimefix, how='left')
ufodf = ufodf.rename(columns={0: "year"})

In [10]:
ufodf = ufodf.drop(ufodf[ufodf['year'] <= 2010].index)
ufodf = ufodf.drop('year', axis = 1)
ufodf = ufodf.rename(columns={'duration (seconds)':'duration_seconds',
                              'duration (hours/min)':'duration_hours_min', 
                              'date posted':'date_posted'})
ufodf['state'] = ufodf['state'].str.upper()
ufodf = ufodf.reset_index()
ufodf = ufodf.rename_axis('id').drop('index', axis = 1)
statecheckerdf = pd.DataFrame(clean_urban["State"])
ufodf = statecheckerdf.merge(ufodf, left_on="State", right_on="state", how="left").drop('State', axis=1)
ufodf.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,10/10/2012 20:00,moundville,AL,us,changing,240,4 minutes,Orange fiery lighting objects changing locations.,10/30/2012,32.9975000,-87.630000
1,10/13/2011 20:38,gurley,AL,us,diamond,480,6 to 8 mins,Super bright star shaped light&#44 desending s...,10/19/2011,34.7016667,-86.375833
2,10/13/2012 06:30,cedar bluff,AL,us,circle,240,3-4 minutes,Bright star like ufo.,10/30/2012,34.2200000,-85.607778
3,10/14/2011 20:35,eufaula,AL,us,light,25,25 secs,A large extremely bright silent white light mo...,10/25/2011,31.8911111,-85.145556
4,10/15/2011 19:00,toney,AL,us,circle,1800,less than 30 mins.,Several orb like thing 5 white 1 red,1/12/2012,34.8980556,-86.733611


**Create Population Table**

In [11]:
states = "Resources/nst-est2019-alldata.csv"
statesdf = pd.read_csv(states, low_memory=False)

In [12]:
statespopdf1 = statesdf[["REGION", "DIVISION", "NAME"]]

In [13]:
statespopdf2 = statesdf[["POPESTIMATE2010", 
                         "POPESTIMATE2011", "POPESTIMATE2012", "POPESTIMATE2013", 
                         "POPESTIMATE2014", "POPESTIMATE2014", "POPESTIMATE2015", 
                         "POPESTIMATE2016", "POPESTIMATE2017", "POPESTIMATE2018",
                        "POPESTIMATE2019"]]


In [14]:
statespopdf2['avgpop'] = statespopdf2.mean(axis=1).astype('int64')

C:\Users\kelly\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
statespopdf = statespopdf1.join(statespopdf2)
statespopdf = statespopdf[["REGION", "DIVISION", "NAME", "avgpop"]]

In [16]:
statespopdf['abbrev'] = statespopdf.NAME.replace(state_new)
statespopdf = statespopdf.iloc[5:-1]

In [17]:
statespopdf = statespopdf[['abbrev', 'avgpop']].copy()
statespopdf = statespopdf.rename(columns={'abbrev':'State','avgpop':'Population'})
statespopdf.head()

,State,Population
5,AL,4844999
6,AK,732859
7,AZ,6798170
8,AR,2973257
9,CA,38616261


**Assign States to Regions**

In [18]:
census = "Resources/nst-est2019-alldata.csv"
censusdf = pd.read_csv(census, low_memory=False)
censusdf = pd.DataFrame(censusdf)

In [19]:
ceny = censusdf[["REGION", "DIVISION", "NAME"]]
ceny = ceny.rename(columns={'NAME':'State'})
ceny = ceny.drop(index = [0,1,2,3,4,56])

In [20]:
ceny["REGION"] = pd.to_numeric(ceny["REGION"])
ceny["DIVISION"] = pd.to_numeric(ceny["DIVISION"])

In [21]:
Regions = [
    (ceny['REGION'] <= 1),
    (ceny['REGION'] <= 2),
    (ceny['REGION'] <= 3),
    (ceny['REGION'] <= 4)
    ]
values = ['Northeast', 'Midwest', 'South', 'West']
ceny['Regions'] = np.select(Regions, values)
Divisions = [
    (ceny['DIVISION'] <= 1),
    (ceny['DIVISION'] <= 2),
    (ceny['DIVISION'] <= 3),
    (ceny['DIVISION'] <= 4),
    (ceny['DIVISION'] <= 5),
    (ceny['DIVISION'] <= 6),
    (ceny['DIVISION'] <= 7),
    (ceny['DIVISION'] <= 8),
    (ceny['DIVISION'] <= 9),
    ]
names = ['New England', 'Middle Atlantic', 'East North Central', 
          'West North Central','South Atlantic', 'East South Central',
          'West South Central', 'Mountain', 'Pacific']
ceny['Divisions'] = np.select(Divisions, names)

In [22]:
census_regions = ceny[["State", "Regions", "Divisions"]]
census_regions = census_regions.rename(columns={"Regions":'Region',"Divisions":"Division"})
census_regions['State'] = census_regions.State.replace(state_new)
census_regions = census_regions.set_index(['State'])
census_regions.head()

,Region,Division
State,,
AL,South,East South Central
AK,West,Pacific
AZ,West,Mountain
AR,South,West South Central
CA,West,Pacific


**Turn New Tables into CSVs**

In [23]:
clean_urban.to_csv("Resources/clean_urban.csv", index=False)
ufodf.to_csv("Resources/limited_ufo.csv", index=True)
statespopdf.to_csv("Resources/state_populations.csv", index=False)
census_regions.to_csv("Resources/census_regions.csv")